# Importing Packages

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model #GARCH Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import math
import scipy.stats
from statistics import NormalDist
from scipy.stats import t
from scipy.stats import f
from datetime import datetime
from scipy.stats import shapiro #Shapiro Test
from scipy import stats #t-test
import statsmodels.api as sm #Ljung-Box Test
from scipy.stats import chi2

# Price and Returns

In [61]:
start_date = '2006-01-01' #yyyy-mm-dd
end_date = '2021-01-01'


# Thailand
MY = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/2006-2021/%5EKLSE.csv')
MY['Date'] = pd.to_datetime(MY['Date'])
MY = MY.rename(columns={'Price': 'Close'})
MY = MY.replace(',','', regex=True)
MY['Close'] = MY['Close'].astype(float, errors = 'raise')
mask = (MY['Date'] >= start_date) & (MY['Date'] <= end_date)
MY = MY.loc[mask]
MY = MY.set_index('Date')
MY = MY.sort_index(axis=0, ascending = False)

In [62]:
def get_returns(df,d):
    '''
    The function obtains the log returns of the asset shifted d days
    
    PARAMETERS
    ----------
    df : pandas.DataFrame
        The data frame contains data of a chosen stock index 
        (Stock index must be arrange in DESCENDING ORDER by DATE)
    d : int
        The dth day being forecast
        (Assumed to be 1 for most cases)
        
    RETURNS
    -------
    df : pandas.DataFrame
        The data frame returns an updated data frame containing the
        'Returns' column
    '''
    df['Previous'] = df['Close'].shift(-d)
    df['Returns'] = np.log(df['Close']/df['Previous'])*100
    display(df)
    return df

MY = get_returns(MY, 1)

,Open,High,Low,Close,Previous,Returns
Date,,,,,,
2020-12-31,1627.21,1627.21,1627.21,1627.21,1644.41,-1.051477
2020-12-30,1644.41,1644.41,1644.41,1644.41,1634.99,0.574497
2020-12-29,1634.99,1634.99,1634.99,1634.99,1643.90,-0.543478
2020-12-28,1643.90,1643.90,1643.90,1643.90,1641.17,0.166207
2020-12-24,1641.17,1641.17,1641.17,1641.17,1647.50,-0.384959
...,...,...,...,...,...,...
2006-01-09,913.65,916,912.4,913.80,911.67,0.233365
2006-01-06,906.6,912.07,906.6,911.67,906.66,0.551056
2006-01-05,897.34,908.71,897.34,906.66,897.13,1.056674


# Diebold-Mariano Test

In [63]:
# Author   : John Tsang
# Date     : December 7th, 2017
# Purpose  : Implement the Diebold-Mariano Test (DM test) to compare 
#            forecast accuracy
# Input    : 1) actual_lst: the list of actual values
#            2) pred1_lst : the first list of predicted values
#            3) pred2_lst : the second list of predicted values
#            4) h         : the number of stpes ahead
#            5) crit      : a string specifying the criterion 
#                             i)  MSE : the mean squared error
#                            ii)  MAD : the mean absolute deviation
#                           iii) MAPE : the mean absolute percentage error
#                            iv) poly : use power function to weigh the errors
#            6) poly      : the power for crit power 
#                           (it is only meaningful when crit is "poly")
# Condition: 1) length of actual_lst, pred1_lst and pred2_lst is equal
#            2) h must be an integer and it must be greater than 0 and less than 
#               the length of actual_lst.
#            3) crit must take the 4 values specified in Input
#            4) Each value of actual_lst, pred1_lst and pred2_lst must
#               be numerical values. Missing values will not be accepted.
#            5) power must be a numerical value.
# Return   : a named-tuple of 2 elements
#            1) p_value : the p-value of the DM test
#            2) DM      : the test statistics of the DM test
##########################################################
# References:
#
# Harvey, D., Leybourne, S., & Newbold, P. (1997). Testing the equality of 
#   prediction mean squared errors. International Journal of forecasting, 
#   13(2), 281-291.
#
# Diebold, F. X. and Mariano, R. S. (1995), Comparing predictive accuracy, 
#   Journal of business & economic statistics 13(3), 253-264.
#
##########################################################
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

## Custom DM-Test Function

In [64]:
def DM_test(country, model_1, model_2, t):
    
    arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
    arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()

    arr_model_2_dropped = model_2.drop(['Date'], axis = 1)
    arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()

    returns = np.array(country['Returns'])[:t]

    dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')
    
    return dm

# Fixed Window

## 0.01%

In [65]:
MY_fw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/MY_VaR_FW-DN-0.01%25.csv')
MY_fw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/MY_VaR_FW-HS-0.01%25.csv')
MY_fw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/MY_VaR_FW-BRW-0.01%25.csv')
MY_fw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/MY_VaR_FW-G-0.01%25.csv')
MY_fw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/MY_VaR_FW-EA-0.01%25.csv')
MY_fw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/MY_VaR_FW-E-0.01%25.csv')
MY_fw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-FW/MY_fw_FW-I-0.01%25.csv')
MY_fw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-FW/MY_fw_FW-FI-0.01%25.csv')
MY_fw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-FW/MY_fw_FW-FI0.633-0.01%25.csv')

In [66]:
# DN
print(DM_test(MY, MY_fw_01_dn, MY_fw_01_garch, len(MY_fw_01_dn)))
print(DM_test(MY, MY_fw_01_dn, MY_fw_01_earch, len(MY_fw_01_dn)))
print(DM_test(MY, MY_fw_01_dn, MY_fw_01_egarch, len(MY_fw_01_dn)))
print(DM_test(MY, MY_fw_01_dn, MY_fw_01_igarch, len(MY_fw_01_dn)))
print(DM_test(MY, MY_fw_01_dn, MY_fw_01_figarch, len(MY_fw_01_dn)))
print(DM_test(MY, MY_fw_01_dn, MY_fw_01_figarch0633, len(MY_fw_01_dn)))

dm_return(DM=-5.525412369680618, p_value=4.4323113118483914e-08)
dm_return(DM=-2.0499571178367884, p_value=0.040690081416670326)
dm_return(DM=-6.234907403302782, p_value=7.270368144696863e-10)
dm_return(DM=-3.749227227198841, p_value=0.00018999415318227895)
dm_return(DM=-5.914485454594939, p_value=4.907432223255662e-09)
dm_return(DM=-5.2281056616391455, p_value=2.1811868616310432e-07)


In [67]:
# HS
print(DM_test(MY, MY_fw_01_hs, MY_fw_01_garch, len(MY_fw_01_hs)))
print(DM_test(MY, MY_fw_01_hs, MY_fw_01_earch, len(MY_fw_01_hs)))
print(DM_test(MY, MY_fw_01_hs, MY_fw_01_egarch, len(MY_fw_01_hs)))
print(DM_test(MY, MY_fw_01_hs, MY_fw_01_igarch, len(MY_fw_01_hs)))
print(DM_test(MY, MY_fw_01_hs, MY_fw_01_figarch, len(MY_fw_01_hs)))
print(DM_test(MY, MY_fw_01_hs, MY_fw_01_figarch0633, len(MY_fw_01_hs)))

dm_return(DM=12.40370902941689, p_value=1.80088318418745e-32)
dm_return(DM=47.15149160396306, p_value=1.8952846619270728e-234)
dm_return(DM=17.247876507737285, p_value=5.134304106763176e-57)
dm_return(DM=6.113222726329844, p_value=1.516866604631002e-09)
dm_return(DM=12.99713072491121, p_value=3.231524348673687e-35)
dm_return(DM=9.304605060738673, p_value=1.2274627430428731e-19)


In [68]:
# BRW
print(DM_test(MY, MY_fw_01_brw, MY_fw_01_garch, len(MY_fw_01_brw)))
print(DM_test(MY, MY_fw_01_brw, MY_fw_01_earch, len(MY_fw_01_brw)))
print(DM_test(MY, MY_fw_01_brw, MY_fw_01_egarch, len(MY_fw_01_brw)))
print(DM_test(MY, MY_fw_01_brw, MY_fw_01_igarch, len(MY_fw_01_brw)))
print(DM_test(MY, MY_fw_01_brw, MY_fw_01_figarch, len(MY_fw_01_brw)))
print(DM_test(MY, MY_fw_01_brw, MY_fw_01_figarch0633, len(MY_fw_01_brw)))

dm_return(DM=17.35808824028933, p_value=1.2699636141960352e-57)
dm_return(DM=21.897526260917477, p_value=7.5342833915063485e-84)
dm_return(DM=19.66908236387749, p_value=1.0439609836989165e-70)
dm_return(DM=11.28589073605775, p_value=1.5482962902988635e-27)
dm_return(DM=17.90435179469565, p_value=1.1790319904884121e-60)
dm_return(DM=14.91421840087893, p_value=1.2623648596708995e-44)


In [69]:
# GARCH
print(DM_test(MY, MY_fw_01_garch, MY_fw_01_earch, len(MY_fw_01_garch)))
print(DM_test(MY, MY_fw_01_garch, MY_fw_01_egarch, len(MY_fw_01_garch)))
print(DM_test(MY, MY_fw_01_garch, MY_fw_01_igarch, len(MY_fw_01_garch)))
print(DM_test(MY, MY_fw_01_garch, MY_fw_01_figarch, len(MY_fw_01_garch)))
print(DM_test(MY, MY_fw_01_garch, MY_fw_01_figarch0633, len(MY_fw_01_garch)))

dm_return(DM=5.365954989769935, p_value=1.0518619208919475e-07)
dm_return(DM=2.932133596758134, p_value=0.0034612480971045376)
dm_return(DM=-0.9818954376902551, p_value=0.32644454736741946)
dm_return(DM=-0.43849436966163285, p_value=0.6611448751145692)
dm_return(DM=-1.8176631306107225, p_value=0.06948495464049079)


In [70]:
# EARCH
print(DM_test(MY, MY_fw_01_earch, MY_fw_01_egarch, len(MY_fw_01_earch)))
print(DM_test(MY, MY_fw_01_earch, MY_fw_01_igarch, len(MY_fw_01_earch)))
print(DM_test(MY, MY_fw_01_earch, MY_fw_01_figarch, len(MY_fw_01_earch)))
print(DM_test(MY, MY_fw_01_earch, MY_fw_01_figarch0633, len(MY_fw_01_earch)))

dm_return(DM=-5.92089123046178, p_value=4.727663709761761e-09)
dm_return(DM=-3.929162565619857, p_value=9.251554037802072e-05)
dm_return(DM=-5.993599101374904, p_value=3.0875777522589706e-09)
dm_return(DM=-5.501714790043318, p_value=5.046783451369176e-08)


In [71]:
# EGARCH
print(DM_test(MY, MY_fw_01_egarch, MY_fw_01_igarch, len(MY_fw_01_egarch)))
print(DM_test(MY, MY_fw_01_egarch, MY_fw_01_figarch, len(MY_fw_01_egarch)))
print(DM_test(MY, MY_fw_01_egarch, MY_fw_01_figarch0633, len(MY_fw_01_egarch)))

dm_return(DM=-1.4589761360596414, p_value=0.14495933989763735)
dm_return(DM=-3.3019052600671372, p_value=0.0010023715682878384)
dm_return(DM=-2.642345469474145, p_value=0.008392317778384015)


In [72]:
# IGARCH
print(DM_test(MY, MY_fw_01_igarch, MY_fw_01_figarch, len(MY_fw_01_igarch)))
print(DM_test(MY, MY_fw_01_igarch, MY_fw_01_figarch0633, len(MY_fw_01_igarch)))

dm_return(DM=1.0027949642008394, p_value=0.31625926947596816)
dm_return(DM=0.4350015024370884, p_value=0.6636772097634175)


In [73]:
# FIGARCH
print(DM_test(MY, MY_fw_01_figarch, MY_fw_01_figarch0633, len(MY_fw_01_figarch)))

dm_return(DM=-2.0702901197453363, p_value=0.038741555308213375)


## 1.00 %

In [74]:
MY_fw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/MY_VaR_FW-DN-1%25.csv')
MY_fw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/MY_VaR_FW-HS-1%25.csv')
MY_fw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/MY_VaR_FW-BRW-1%25.csv')
MY_fw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/MY_VaR_FW-G-1%25.csv')
MY_fw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/MY_VaR_FW-EA-1%25.csv')
MY_fw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/MY_VaR_FW-E-1%25.csv')
MY_fw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-FW/MY_fw_FW-I-1%25.csv')
MY_fw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-FW/MY_fw_FW-FI-1%25.csv')
MY_fw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-FW/MY_fw_FW-FI0.633-1%25.csv')

In [75]:
# DN
print(DM_test(MY, MY_fw_1_dn, MY_fw_1_garch, len(MY_fw_1_dn)))
print(DM_test(MY, MY_fw_1_dn, MY_fw_1_earch, len(MY_fw_1_dn)))
print(DM_test(MY, MY_fw_1_dn, MY_fw_1_egarch, len(MY_fw_1_dn)))
print(DM_test(MY, MY_fw_1_dn, MY_fw_1_igarch, len(MY_fw_1_dn)))
print(DM_test(MY, MY_fw_1_dn, MY_fw_1_figarch, len(MY_fw_1_dn)))
print(DM_test(MY, MY_fw_1_dn, MY_fw_1_figarch0633, len(MY_fw_1_dn)))

dm_return(DM=-5.484817526721043, p_value=5.5346473663794396e-08)
dm_return(DM=-2.0643889703460028, p_value=0.03929871251303416)
dm_return(DM=-6.188368779410073, p_value=9.646063871072508e-10)
dm_return(DM=-3.7478080386666495, p_value=0.0001910535429526348)
dm_return(DM=-5.860613844940082, p_value=6.707438591399449e-09)
dm_return(DM=-5.193688271316534, p_value=2.6100610774832687e-07)


In [76]:
# HS
print(DM_test(MY, MY_fw_1_hs, MY_fw_1_garch, len(MY_fw_1_hs)))
print(DM_test(MY, MY_fw_1_hs, MY_fw_1_earch, len(MY_fw_1_hs)))
print(DM_test(MY, MY_fw_1_hs, MY_fw_1_egarch, len(MY_fw_1_hs)))
print(DM_test(MY, MY_fw_1_hs, MY_fw_1_igarch, len(MY_fw_1_hs)))
print(DM_test(MY, MY_fw_1_hs, MY_fw_1_figarch, len(MY_fw_1_hs)))
print(DM_test(MY, MY_fw_1_hs, MY_fw_1_figarch0633, len(MY_fw_1_hs)))

dm_return(DM=-2.190487300384025, p_value=0.028773267848795678)
dm_return(DM=8.212782348871944, p_value=8.509290372081158e-16)
dm_return(DM=-1.8884120089356413, p_value=0.05932776534524013)
dm_return(DM=-1.9127985967023489, p_value=0.05612620567157285)
dm_return(DM=-2.3890355061988897, p_value=0.017120599468267845)
dm_return(DM=-2.514799132274283, p_value=0.01210265150046848)


In [77]:
# BRW
print(DM_test(MY, MY_fw_1_brw, MY_fw_1_garch, len(MY_fw_1_brw)))
print(DM_test(MY, MY_fw_1_brw, MY_fw_1_earch, len(MY_fw_1_brw)))
print(DM_test(MY, MY_fw_1_brw, MY_fw_1_egarch, len(MY_fw_1_brw)))
print(DM_test(MY, MY_fw_1_brw, MY_fw_1_igarch, len(MY_fw_1_brw)))
print(DM_test(MY, MY_fw_1_brw, MY_fw_1_figarch, len(MY_fw_1_brw)))
print(DM_test(MY, MY_fw_1_brw, MY_fw_1_figarch0633, len(MY_fw_1_brw)))

dm_return(DM=-1.1654106707344516, p_value=0.24419586940779855)
dm_return(DM=7.239399267134809, p_value=1.0480581558944753e-12)
dm_return(DM=-0.5359949778920454, p_value=0.5921091719748086)
dm_return(DM=-1.3154606996565519, p_value=0.18872734229193827)
dm_return(DM=-1.3180976090492509, p_value=0.1878435988518311)
dm_return(DM=-1.6626571553040164, p_value=0.09676795511714432)


In [78]:
# GARCH
print(DM_test(MY, MY_fw_1_garch, MY_fw_1_earch, len(MY_fw_1_garch)))
print(DM_test(MY, MY_fw_1_garch, MY_fw_1_egarch, len(MY_fw_1_garch)))
print(DM_test(MY, MY_fw_1_garch, MY_fw_1_igarch, len(MY_fw_1_garch)))
print(DM_test(MY, MY_fw_1_garch, MY_fw_1_figarch, len(MY_fw_1_garch)))
print(DM_test(MY, MY_fw_1_garch, MY_fw_1_figarch0633, len(MY_fw_1_garch)))

dm_return(DM=5.305623696302467, p_value=1.4503252394198996e-07)
dm_return(DM=2.8623662220691943, p_value=0.0043133536577769195)
dm_return(DM=-0.9793767660647633, p_value=0.3276862704304383)
dm_return(DM=-0.4184506068669573, p_value=0.6757286136614769)
dm_return(DM=-1.8120445656133954, p_value=0.0703494946171443)


In [79]:
# EARCH
print(DM_test(MY, MY_fw_1_earch, MY_fw_1_egarch, len(MY_fw_1_earch)))
print(DM_test(MY, MY_fw_1_earch, MY_fw_1_igarch, len(MY_fw_1_earch)))
print(DM_test(MY, MY_fw_1_earch, MY_fw_1_figarch, len(MY_fw_1_earch)))
print(DM_test(MY, MY_fw_1_earch, MY_fw_1_figarch0633, len(MY_fw_1_earch)))

dm_return(DM=-5.850224376622263, p_value=7.1220303652690615e-09)
dm_return(DM=-3.920785703745579, p_value=9.572907529310306e-05)
dm_return(DM=-5.906680499056063, p_value=5.135487253003704e-09)
dm_return(DM=-5.443272644021391, p_value=6.936898732510952e-08)


In [80]:
# EGARCH
print(DM_test(MY, MY_fw_1_egarch, MY_fw_1_igarch, len(MY_fw_1_egarch)))
print(DM_test(MY, MY_fw_1_egarch, MY_fw_1_figarch, len(MY_fw_1_egarch)))
print(DM_test(MY, MY_fw_1_egarch, MY_fw_1_figarch0633, len(MY_fw_1_egarch)))

dm_return(DM=-1.4485263469791814, p_value=0.1478568848287053)
dm_return(DM=-3.2269548963541457, p_value=0.0013014328921285051)
dm_return(DM=-2.6127857547875086, p_value=0.009147239838081326)


In [81]:
# IGARCH
print(DM_test(MY, MY_fw_1_igarch, MY_fw_1_figarch, len(MY_fw_1_igarch)))
print(DM_test(MY, MY_fw_1_igarch, MY_fw_1_figarch0633, len(MY_fw_1_igarch)))

dm_return(DM=1.0020617277137265, p_value=0.3166130329305583)
dm_return(DM=0.43114740138497604, p_value=0.6664759115069575)


In [82]:
# FIGARCH
print(DM_test(MY, MY_fw_1_figarch, MY_fw_1_figarch0633, len(MY_fw_1_figarch)))

dm_return(DM=-2.0663413822107417, p_value=0.0391136257130063)


## 5.00 %

In [83]:
MY_fw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/MY_VaR_FW-DN-5%25.csv')
MY_fw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/MY_VaR_FW-HS-5%25.csv')
MY_fw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/MY_VaR_FW-BRW-5%25.csv')
MY_fw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/MY_VaR_FW-G-5%25.csv')
MY_fw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/MY_VaR_FW-EA-5%25.csv')
MY_fw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/MY_VaR_FW-E-5%25.csv')
MY_fw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-FW/MY_fw_FW-I-5%25.csv')
MY_fw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-FW/MY_fw_FW-FI-5%25.csv')
MY_fw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-FW/MY_fw_FW-FI0.633-5%25.csv')

In [84]:
# DN
print(DM_test(MY, MY_fw_5_dn, MY_fw_5_garch, len(MY_fw_5_dn)))
print(DM_test(MY, MY_fw_5_dn, MY_fw_5_earch, len(MY_fw_5_dn)))
print(DM_test(MY, MY_fw_5_dn, MY_fw_5_egarch, len(MY_fw_5_dn)))
print(DM_test(MY, MY_fw_5_dn, MY_fw_5_igarch, len(MY_fw_5_dn)))
print(DM_test(MY, MY_fw_5_dn, MY_fw_5_figarch, len(MY_fw_5_dn)))
print(DM_test(MY, MY_fw_5_dn, MY_fw_5_figarch0633, len(MY_fw_5_dn)))

dm_return(DM=-5.423504754033424, p_value=7.719753403344474e-08)
dm_return(DM=-2.064970575798991, p_value=0.03924349911209493)
dm_return(DM=-6.11103715368949, p_value=1.536859699894616e-09)
dm_return(DM=-3.7352605947190045, p_value=0.00020066490306094658)
dm_return(DM=-5.782054503340117, p_value=1.0532097171341041e-08)
dm_return(DM=-5.139914205308111, p_value=3.447886201448849e-07)


In [85]:
# HS
print(DM_test(MY, MY_fw_5_hs, MY_fw_5_garch, len(MY_fw_5_hs)))
print(DM_test(MY, MY_fw_5_hs, MY_fw_5_earch, len(MY_fw_5_hs)))
print(DM_test(MY, MY_fw_5_hs, MY_fw_5_egarch, len(MY_fw_5_hs)))
print(DM_test(MY, MY_fw_5_hs, MY_fw_5_igarch, len(MY_fw_5_hs)))
print(DM_test(MY, MY_fw_5_hs, MY_fw_5_figarch, len(MY_fw_5_hs)))
print(DM_test(MY, MY_fw_5_hs, MY_fw_5_figarch0633, len(MY_fw_5_hs)))

dm_return(DM=-7.633451602805647, p_value=6.432105962827574e-14)
dm_return(DM=-9.20692307578089, p_value=2.8064007867537274e-19)
dm_return(DM=-8.940164548171689, p_value=2.5919810500190093e-18)
dm_return(DM=-5.016557485118146, p_value=6.467654514602463e-07)
dm_return(DM=-8.099048313922426, p_value=2.0304892432533394e-15)
dm_return(DM=-6.960707038747408, p_value=6.995277167679922e-12)


In [86]:
# BRW
print(DM_test(MY, MY_fw_5_brw, MY_fw_5_garch, len(MY_fw_5_brw)))
print(DM_test(MY, MY_fw_5_brw, MY_fw_5_earch, len(MY_fw_5_brw)))
print(DM_test(MY, MY_fw_5_brw, MY_fw_5_egarch, len(MY_fw_5_brw)))
print(DM_test(MY, MY_fw_5_brw, MY_fw_5_igarch, len(MY_fw_5_brw)))
print(DM_test(MY, MY_fw_5_brw, MY_fw_5_figarch, len(MY_fw_5_brw)))
print(DM_test(MY, MY_fw_5_brw, MY_fw_5_figarch0633, len(MY_fw_5_brw)))

dm_return(DM=-5.223340800973988, p_value=2.2362090544404198e-07)
dm_return(DM=-1.2953928963810346, p_value=0.19555378178219412)
dm_return(DM=-5.933798822283045, p_value=4.384720558263237e-09)
dm_return(DM=-3.6373202868130945, p_value=0.00029292091527519554)
dm_return(DM=-5.637963869125127, p_value=2.3765965422583078e-08)
dm_return(DM=-4.989746118335347, p_value=7.402156499086282e-07)


In [87]:
# GARCH
print(DM_test(MY, MY_fw_5_garch, MY_fw_5_earch, len(MY_fw_5_garch)))
print(DM_test(MY, MY_fw_5_garch, MY_fw_5_egarch, len(MY_fw_5_garch)))
print(DM_test(MY, MY_fw_5_garch, MY_fw_5_igarch, len(MY_fw_5_garch)))
print(DM_test(MY, MY_fw_5_garch, MY_fw_5_figarch, len(MY_fw_5_garch)))
print(DM_test(MY, MY_fw_5_garch, MY_fw_5_figarch0633, len(MY_fw_5_garch)))

dm_return(DM=5.223978658171484, p_value=2.2287662549355012e-07)
dm_return(DM=2.778931044025306, p_value=0.005580095738122065)
dm_return(DM=-0.9735462748051315, p_value=0.3305725109748485)
dm_return(DM=-0.3948925654646945, p_value=0.6930261381164149)
dm_return(DM=-1.7988406916134556, p_value=0.07241603854868238)


In [88]:
# EARCH
print(DM_test(MY, MY_fw_5_earch, MY_fw_5_egarch, len(MY_fw_5_earch)))
print(DM_test(MY, MY_fw_5_earch, MY_fw_5_igarch, len(MY_fw_5_earch)))
print(DM_test(MY, MY_fw_5_earch, MY_fw_5_figarch, len(MY_fw_5_earch)))
print(DM_test(MY, MY_fw_5_earch, MY_fw_5_figarch0633, len(MY_fw_5_earch)))

dm_return(DM=-5.748351989011691, p_value=1.276080620839513e-08)
dm_return(DM=-3.902558644787867, p_value=0.00010308989352569006)
dm_return(DM=-5.79365107423206, p_value=9.856730055653814e-09)
dm_return(DM=-5.366356785640266, p_value=1.049602930539289e-07)


In [89]:
# EGARCH
print(DM_test(MY, MY_fw_5_egarch, MY_fw_5_igarch, len(MY_fw_5_egarch)))
print(DM_test(MY, MY_fw_5_egarch, MY_fw_5_figarch, len(MY_fw_5_egarch)))
print(DM_test(MY, MY_fw_5_egarch, MY_fw_5_figarch0633, len(MY_fw_5_egarch)))

dm_return(DM=-1.4329179697279764, p_value=0.15226711285148864)
dm_return(DM=-3.1371006692987224, p_value=0.0017679835957862642)
dm_return(DM=-2.5729275253112025, p_value=0.010261165634800899)


In [90]:
# IGARCH
print(DM_test(MY, MY_fw_5_igarch, MY_fw_5_figarch,  len(MY_fw_5_igarch)))
print(DM_test(MY, MY_fw_5_igarch, MY_fw_5_figarch0633, len(MY_fw_5_igarch)))

dm_return(DM=0.9978909671779277, p_value=0.3186302392546757)
dm_return(DM=0.4257080089955658, p_value=0.670433711517192)


In [91]:
# FIGARCH
print(DM_test(MY, MY_fw_5_figarch, MY_fw_5_figarch0633, len(MY_fw_5_figarch)))

dm_return(DM=-2.053145752341633, p_value=0.040379118083651036)


# Rolling Window

## 0.01 %

In [92]:
MY_rw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/MY_VaR_RW-DN-0.01%25.csv')
MY_rw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/MY_VaR_RW-HS-0.01%25.csv')
MY_rw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/MY_VaR_RW-BRW-0.01%25.csv')
MY_rw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/MY_VaR_RW-G-0.01%25.csv')
MY_rw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/MY_VaR_RW-EA-0.01%25.csv')
MY_rw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/MY_VaR_RW-E-0.01%25.csv')
MY_rw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-RW/MY_RW-I-0.01%25.csv')
MY_rw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-RW/MY_RW-FI-0.01%25.csv')
MY_rw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-RW/MY_RW-FI0.633-0.01%25.csv')

In [93]:
# DN
print(DM_test(MY, MY_rw_01_dn, MY_rw_01_garch, len(MY_rw_01_dn)))
print(DM_test(MY, MY_rw_01_dn, MY_rw_01_earch, len(MY_rw_01_dn)))
print(DM_test(MY, MY_rw_01_dn, MY_rw_01_egarch, len(MY_rw_01_dn)))
print(DM_test(MY, MY_rw_01_dn, MY_rw_01_igarch, len(MY_rw_01_dn)))
print(DM_test(MY, MY_rw_01_dn, MY_rw_01_figarch, len(MY_rw_01_dn)))
print(DM_test(MY, MY_rw_01_dn, MY_rw_01_figarch0633, len(MY_rw_01_dn)))

dm_return(DM=-4.736394610995147, p_value=2.5682538311172374e-06)
dm_return(DM=-1.7571937191487639, p_value=0.07926256034167718)
dm_return(DM=-5.232020351652553, p_value=2.1369648593588935e-07)
dm_return(DM=-3.2586570842062232, p_value=0.001166074819057097)
dm_return(DM=-4.9520758636938575, p_value=8.938087959272899e-07)
dm_return(DM=-4.520263291257793, p_value=7.097071945782543e-06)


In [94]:
# HS
print(DM_test(MY, MY_rw_01_hs, MY_rw_01_garch, len(MY_rw_01_hs)))
print(DM_test(MY, MY_rw_01_hs, MY_rw_01_earch, len(MY_rw_01_hs)))
print(DM_test(MY, MY_rw_01_hs, MY_rw_01_egarch, len(MY_rw_01_hs)))
print(DM_test(MY, MY_rw_01_hs, MY_rw_01_igarch, len(MY_rw_01_hs)))
print(DM_test(MY, MY_rw_01_hs, MY_rw_01_figarch, len(MY_rw_01_hs)))
print(DM_test(MY, MY_rw_01_hs, MY_rw_01_figarch0633, len(MY_rw_01_hs)))

dm_return(DM=35.18942198658796, p_value=2.39293360789654e-165)
dm_return(DM=48.79693375476316, p_value=2.2350202574323693e-243)
dm_return(DM=41.89702393838485, p_value=5.76805608659289e-205)
dm_return(DM=21.758362900006713, p_value=5.121638051978146e-83)
dm_return(DM=36.69872072630339, p_value=1.9644780908407013e-174)
dm_return(DM=29.958097012937486, p_value=2.6204179243222235e-133)


In [95]:
# BRW
print(DM_test(MY, MY_rw_01_brw, MY_rw_01_garch, len(MY_rw_01_brw)))
print(DM_test(MY, MY_rw_01_brw, MY_rw_01_earch, len(MY_rw_01_brw)))
print(DM_test(MY, MY_rw_01_brw, MY_rw_01_egarch, len(MY_rw_01_brw)))
print(DM_test(MY, MY_rw_01_brw, MY_rw_01_igarch, len(MY_rw_01_brw)))
print(DM_test(MY, MY_rw_01_brw, MY_rw_01_figarch, len(MY_rw_01_brw)))
print(DM_test(MY, MY_rw_01_brw, MY_rw_01_figarch0633, len(MY_rw_01_brw)))

dm_return(DM=16.976177323172614, p_value=1.5795295229701136e-55)
dm_return(DM=21.641507073706677, p_value=2.5544105308977236e-82)
dm_return(DM=19.365751255160696, p_value=5.914934991406182e-69)
dm_return(DM=11.191328558536346, p_value=3.9105921080549465e-27)
dm_return(DM=17.514292913700206, p_value=1.741584123795807e-58)
dm_return(DM=14.712902179580226, p_value=1.333041201474072e-43)


In [96]:
# GARCH
print(DM_test(MY, MY_rw_01_garch, MY_rw_01_earch, len(MY_rw_01_garch)))
print(DM_test(MY, MY_rw_01_garch, MY_rw_01_egarch, len(MY_rw_01_garch)))
print(DM_test(MY, MY_rw_01_garch, MY_rw_01_igarch, len(MY_rw_01_garch)))
print(DM_test(MY, MY_rw_01_garch, MY_rw_01_figarch, len(MY_rw_01_garch)))
print(DM_test(MY, MY_rw_01_garch, MY_rw_01_figarch0633, len(MY_rw_01_garch)))

dm_return(DM=4.584776533507256, p_value=5.262534116702498e-06)
dm_return(DM=2.9128177506851403, p_value=0.0036803149733107563)
dm_return(DM=-0.8360949600940504, p_value=0.4033480551697741)
dm_return(DM=0.28684005151169517, p_value=0.7743081502492306)
dm_return(DM=-1.5807044500269736, p_value=0.11433601628693907)


In [97]:
# EARCH
print(DM_test(MY, MY_rw_01_earch, MY_rw_01_egarch, len(MY_rw_01_earch)))
print(DM_test(MY, MY_rw_01_earch, MY_rw_01_igarch, len(MY_rw_01_earch)))
print(DM_test(MY, MY_rw_01_earch, MY_rw_01_figarch, len(MY_rw_01_earch)))
print(DM_test(MY, MY_rw_01_earch, MY_rw_01_figarch0633, len(MY_rw_01_earch)))

dm_return(DM=-4.915430094446551, p_value=1.072467296260525e-06)
dm_return(DM=-3.4314562888483486, p_value=0.0006307638413159001)
dm_return(DM=-4.958495935399715, p_value=8.656197819226686e-07)
dm_return(DM=-4.758723047797836, p_value=2.3067985115528477e-06)


In [98]:
# EGARCH
print(DM_test(MY, MY_rw_01_egarch, MY_rw_01_igarch, len(MY_rw_01_egarch)))
print(DM_test(MY, MY_rw_01_egarch, MY_rw_01_figarch, len(MY_rw_01_egarch)))
print(DM_test(MY, MY_rw_01_egarch, MY_rw_01_figarch0633, len(MY_rw_01_egarch)))

dm_return(DM=-1.3491278007151986, p_value=0.17767292213680624)
dm_return(DM=-2.922777051689433, p_value=0.003565833959936096)
dm_return(DM=-2.5104160037856045, p_value=0.01225273318825574)


In [99]:
# IGARCH
print(DM_test(MY, MY_rw_01_igarch, MY_rw_01_figarch,  len(MY_rw_01_igarch)))
print(DM_test(MY, MY_rw_01_igarch, MY_rw_01_figarch0633, len(MY_rw_01_igarch)))

dm_return(DM=0.9434780053961925, p_value=0.3457177804081404)
dm_return(DM=0.3687433356053475, p_value=0.7124154018883102)


In [100]:
# FIGARCH
print(DM_test(MY, MY_rw_01_figarch, MY_rw_01_figarch0633, len(MY_rw_01_figarch)))

dm_return(DM=-2.087964830942465, p_value=0.03711286206460161)


## 1.00 %

In [101]:
MY_rw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/MY_VaR_RW-DN-1%25.csv')
MY_rw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/MY_VaR_RW-HS-1%25.csv')
MY_rw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/MY_VaR_RW-BRW-1%25.csv')
MY_rw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/MY_VaR_RW-G-1%25.csv')
MY_rw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/MY_VaR_RW-EA-1%25.csv')
MY_rw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/MY_VaR_RW-E-1%25.csv')
MY_rw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-RW/MY_RW-I-1%25.csv')
MY_rw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-RW/MY_RW-FI-1%25.csv')
MY_rw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-RW/MY_RW-FI0.633-1%25.csv')

In [102]:
# DN
print(DM_test(MY, MY_rw_1_dn, MY_rw_1_garch, len(MY_rw_1_dn)))
print(DM_test(MY, MY_rw_1_dn, MY_rw_1_earch, len(MY_rw_1_dn)))
print(DM_test(MY, MY_rw_1_dn, MY_rw_1_egarch, len(MY_rw_1_dn)))
print(DM_test(MY, MY_rw_1_dn, MY_rw_1_igarch, len(MY_rw_1_dn)))
print(DM_test(MY, MY_rw_1_dn, MY_rw_1_figarch, len(MY_rw_1_dn)))
print(DM_test(MY, MY_rw_1_dn, MY_rw_1_figarch0633, len(MY_rw_1_dn)))

dm_return(DM=-4.714605687130766, p_value=2.8507235415598255e-06)
dm_return(DM=-1.7760561413576974, p_value=0.0760990033719663)
dm_return(DM=-5.216588821058602, p_value=2.3164841133292203e-07)
dm_return(DM=-3.262500359015325, p_value=0.001150582233654387)
dm_return(DM=-4.921250551065271, p_value=1.0419541173241841e-06)
dm_return(DM=-4.499455137473856, p_value=7.809622867517921e-06)


In [103]:
# HS
print(DM_test(MY, MY_rw_1_hs, MY_rw_1_garch, len(MY_rw_1_hs)))
print(DM_test(MY, MY_rw_1_hs, MY_rw_1_earch, len(MY_rw_1_hs)))
print(DM_test(MY, MY_rw_1_hs, MY_rw_1_egarch, len(MY_rw_1_hs)))
print(DM_test(MY, MY_rw_1_hs, MY_rw_1_igarch, len(MY_rw_1_hs)))
print(DM_test(MY, MY_rw_1_hs, MY_rw_1_figarch, len(MY_rw_1_hs)))
print(DM_test(MY, MY_rw_1_hs, MY_rw_1_figarch0633, len(MY_rw_1_hs)))

dm_return(DM=-1.497870110938992, p_value=0.13455658030246273)
dm_return(DM=7.641613006875668, p_value=6.062930867109253e-14)
dm_return(DM=-0.9882307852085978, p_value=0.32333473709620963)
dm_return(DM=-1.4293105046804242, p_value=0.15330055149658217)
dm_return(DM=-1.5286393544328336, p_value=0.12674427247414732)
dm_return(DM=-1.839141501433999, p_value=0.06626024036582963)


In [104]:
# BRW
print(DM_test(MY, MY_rw_1_brw, MY_rw_1_garch, len(MY_rw_1_brw)))
print(DM_test(MY, MY_rw_1_brw, MY_rw_1_earch, len(MY_rw_1_brw)))
print(DM_test(MY, MY_rw_1_brw, MY_rw_1_egarch, len(MY_rw_1_brw)))
print(DM_test(MY, MY_rw_1_brw, MY_rw_1_igarch, len(MY_rw_1_brw)))
print(DM_test(MY, MY_rw_1_brw, MY_rw_1_figarch, len(MY_rw_1_brw)))
print(DM_test(MY, MY_rw_1_brw, MY_rw_1_figarch0633, len(MY_rw_1_brw)))

dm_return(DM=-1.4439820743322045, p_value=0.14913068095417165)
dm_return(DM=5.544044889466521, p_value=4.000821378325854e-08)
dm_return(DM=-0.9051268441579016, p_value=0.36566732990650863)
dm_return(DM=-1.367971163678838, p_value=0.17170047511896283)
dm_return(DM=-1.4748770254586807, p_value=0.14063417399841652)
dm_return(DM=-1.7733658235894523, p_value=0.07654380530607342)


In [105]:
# GARCH
print(DM_test(MY, MY_rw_1_garch, MY_rw_1_earch, len(MY_rw_1_garch)))
print(DM_test(MY, MY_rw_1_garch, MY_rw_1_egarch, len(MY_rw_1_garch)))
print(DM_test(MY, MY_rw_1_garch, MY_rw_1_igarch, len(MY_rw_1_garch)))
print(DM_test(MY, MY_rw_1_garch, MY_rw_1_figarch, len(MY_rw_1_garch)))
print(DM_test(MY, MY_rw_1_garch, MY_rw_1_figarch0633, len(MY_rw_1_garch)))

dm_return(DM=4.54660416968097, p_value=6.284068586847658e-06)
dm_return(DM=2.837211648307612, p_value=0.004664591884550346)
dm_return(DM=-0.8315660418732135, p_value=0.40589900639432086)
dm_return(DM=0.2995490439542717, p_value=0.7645980038743247)
dm_return(DM=-1.5739543866060308, p_value=0.11588848105187381)


In [106]:
# EARCH
print(DM_test(MY, MY_rw_1_earch, MY_rw_1_egarch, len(MY_rw_1_earch)))
print(DM_test(MY, MY_rw_1_earch, MY_rw_1_igarch, len(MY_rw_1_earch)))
print(DM_test(MY, MY_rw_1_earch, MY_rw_1_figarch, len(MY_rw_1_earch)))
print(DM_test(MY, MY_rw_1_earch, MY_rw_1_figarch0633, len(MY_rw_1_earch)))

dm_return(DM=-4.881385932413232, p_value=1.268952151601285e-06)
dm_return(DM=-3.4277032577564555, p_value=0.000639420236967755)
dm_return(DM=-4.901451105285707, p_value=1.149311048367693e-06)
dm_return(DM=-4.714482489233577, p_value=2.8524025374079474e-06)


In [107]:
# EGARCH
print(DM_test(MY, MY_rw_1_egarch, MY_rw_1_igarch, len(MY_rw_1_egarch)))
print(DM_test(MY, MY_rw_1_egarch, MY_rw_1_figarch, len(MY_rw_1_egarch)))
print(DM_test(MY, MY_rw_1_egarch, MY_rw_1_figarch0633, len(MY_rw_1_egarch)))

dm_return(DM=-1.3349553849750488, p_value=0.18226602408539588)
dm_return(DM=-2.8377645251033066, p_value=0.004656601040792582)
dm_return(DM=-2.4745777057107383, p_value=0.013543135536918936)


In [108]:
# IGARCH
print(DM_test(MY, MY_rw_1_igarch, MY_rw_1_figarch,  len(MY_rw_1_igarch)))
print(DM_test(MY, MY_rw_1_igarch, MY_rw_1_figarch0633, len(MY_rw_1_igarch)))

dm_return(DM=0.9399333680141269, p_value=0.3475319474763463)
dm_return(DM=0.3639113387815314, p_value=0.7160191838344072)


In [109]:
# FIGARCH
print(DM_test(MY, MY_rw_1_figarch, MY_rw_1_figarch0633, len(MY_rw_1_figarch)))

dm_return(DM=-2.0806598862705186, p_value=0.03777877903738094)


## 5.00 %

In [110]:
MY_rw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/MY_VaR_RW-DN-5%25.csv')
MY_rw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/MY_VaR_RW-HS-5%25.csv')
MY_rw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/MY_VaR_RW-BRW-5%25.csv')
MY_rw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/MY_VaR_RW-G-5%25.csv')
MY_rw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/MY_VaR_RW-EA-5%25.csv')
MY_rw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/MY_VaR_RW-E-5%25.csv')
MY_rw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-RW/MY_RW-I-5%25.csv')
MY_rw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-RW/MY_RW-FI-5%25.csv')
MY_rw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-RW/MY_RW-FI0.633-5%25.csv')

In [111]:
# DN
print(DM_test(MY, MY_rw_5_dn, MY_rw_5_garch, len(MY_rw_5_dn)))
print(DM_test(MY, MY_rw_5_dn, MY_rw_5_earch, len(MY_rw_5_dn)))
print(DM_test(MY, MY_rw_5_dn, MY_rw_5_egarch, len(MY_rw_5_dn)))
print(DM_test(MY, MY_rw_5_dn, MY_rw_5_igarch, len(MY_rw_5_dn)))
print(DM_test(MY, MY_rw_5_dn, MY_rw_5_figarch, len(MY_rw_5_dn)))
print(DM_test(MY, MY_rw_5_dn, MY_rw_5_figarch0633, len(MY_rw_5_dn)))

dm_return(DM=-4.675655584781827, p_value=3.4317073901933162e-06)
dm_return(DM=-1.7848650731695301, p_value=0.07465734553735913)
dm_return(DM=-5.176930173665618, p_value=2.8473855627401383e-07)
dm_return(DM=-3.2570586132861328, p_value=0.0011725750867887283)
dm_return(DM=-4.871311105224584, p_value=1.3334647752366833e-06)
dm_return(DM=-4.462958360288867, p_value=9.227954872001372e-06)


In [112]:
# HS
print(DM_test(MY, MY_rw_5_hs, MY_rw_5_garch, len(MY_rw_5_hs)))
print(DM_test(MY, MY_rw_5_hs, MY_rw_5_earch, len(MY_rw_5_hs)))
print(DM_test(MY, MY_rw_5_hs, MY_rw_5_egarch, len(MY_rw_5_hs)))
print(DM_test(MY, MY_rw_5_hs, MY_rw_5_igarch, len(MY_rw_5_hs)))
print(DM_test(MY, MY_rw_5_hs, MY_rw_5_figarch, len(MY_rw_5_hs)))
print(DM_test(MY, MY_rw_5_hs, MY_rw_5_figarch0633, len(MY_rw_5_hs)))

dm_return(DM=-7.003722627050593, p_value=5.2402042975938805e-12)
dm_return(DM=-8.769799329122408, p_value=1.0432518589933544e-17)
dm_return(DM=-8.191216908695848, p_value=1.0042532460690007e-15)
dm_return(DM=-4.619826271884677, p_value=4.466398420576519e-06)
dm_return(DM=-7.313763615732387, p_value=6.248672178812616e-13)
dm_return(DM=-6.40185450231783, p_value=2.5974347625751455e-10)


In [113]:
# BRW
print(DM_test(MY, MY_rw_5_brw, MY_rw_5_garch, len(MY_rw_5_brw)))
print(DM_test(MY, MY_rw_5_brw, MY_rw_5_earch, len(MY_rw_5_brw)))
print(DM_test(MY, MY_rw_5_brw, MY_rw_5_egarch, len(MY_rw_5_brw)))
print(DM_test(MY, MY_rw_5_brw, MY_rw_5_igarch, len(MY_rw_5_brw)))
print(DM_test(MY, MY_rw_5_brw, MY_rw_5_figarch, len(MY_rw_5_brw)))
print(DM_test(MY, MY_rw_5_brw, MY_rw_5_figarch0633, len(MY_rw_5_brw)))

dm_return(DM=-5.3865546951994, p_value=9.419120370037476e-08)
dm_return(DM=-3.3686462510007757, p_value=0.0007910663235797386)
dm_return(DM=-6.18162298951108, p_value=1.0048083144357636e-09)
dm_return(DM=-3.6764553677386673, p_value=0.0002520901875923383)
dm_return(DM=-5.659239184751545, p_value=2.1098772426309915e-08)
dm_return(DM=-5.057313990550068, p_value=5.26169324239576e-07)


In [114]:
# GARCH
print(DM_test(MY, MY_rw_5_garch, MY_rw_5_earch, len(MY_rw_5_garch)))
print(DM_test(MY, MY_rw_5_garch, MY_rw_5_egarch, len(MY_rw_5_garch)))
print(DM_test(MY, MY_rw_5_garch, MY_rw_5_igarch, len(MY_rw_5_garch)))
print(DM_test(MY, MY_rw_5_garch, MY_rw_5_figarch, len(MY_rw_5_garch)))
print(DM_test(MY, MY_rw_5_garch, MY_rw_5_figarch0633, len(MY_rw_5_garch)))

dm_return(DM=4.491277545047404, p_value=8.108001456982151e-06)
dm_return(DM=2.7476705900619782, p_value=0.006135419671455707)
dm_return(DM=-0.8242487915507856, p_value=0.4100408482324568)
dm_return(DM=0.31212190544489843, p_value=0.7550282071107318)
dm_return(DM=-1.5617863577820261, p_value=0.11872891930271758)


In [115]:
# EARCH
print(DM_test(MY, MY_rw_5_earch, MY_rw_5_egarch, len(MY_rw_5_earch)))
print(DM_test(MY, MY_rw_5_earch, MY_rw_5_igarch, len(MY_rw_5_earch)))
print(DM_test(MY, MY_rw_5_earch, MY_rw_5_figarch, len(MY_rw_5_earch)))
print(DM_test(MY, MY_rw_5_earch, MY_rw_5_figarch0633, len(MY_rw_5_earch)))

dm_return(DM=-4.82361568360723, p_value=1.6842621390785762e-06)
dm_return(DM=-3.415464026287498, p_value=0.0006684253962873969)
dm_return(DM=-4.824812903697487, p_value=1.6744583862523418e-06)
dm_return(DM=-4.654914812440706, p_value=3.7858872935379722e-06)


In [116]:
# EGARCH
print(DM_test(MY, MY_rw_5_egarch, MY_rw_5_igarch, len(MY_rw_5_egarch)))
print(DM_test(MY, MY_rw_5_egarch, MY_rw_5_figarch, len(MY_rw_5_egarch)))
print(DM_test(MY, MY_rw_5_egarch, MY_rw_5_figarch0633, len(MY_rw_5_egarch)))

dm_return(DM=-1.31589997753306, p_value=0.1885799082445903)
dm_return(DM=-2.7384053477949215, p_value=0.006309333965405733)
dm_return(DM=-2.429360306301861, p_value=0.015341957770308421)


In [117]:
# IGARCH
print(DM_test(MY, MY_rw_5_igarch, MY_rw_5_figarch,  len(MY_rw_5_igarch)))
print(DM_test(MY, MY_rw_5_igarch, MY_rw_5_figarch0633, len(MY_rw_5_igarch)))

dm_return(DM=0.9332165051042648, p_value=0.35098628545263855)
dm_return(DM=0.35760545912793235, p_value=0.7207317501505643)


In [118]:
# FIGARCH
print(DM_test(MY, MY_rw_5_figarch, MY_rw_5_figarch0633, len(MY_rw_5_figarch)))

dm_return(DM=-2.065786341432731, p_value=0.03916616744666534)
